In [156]:
# iMPORT PACKAGES
import pandas as pd
import datetime as dt
import numpy as np

from datetime import timedelta

import matplotlib.pyplot as plt

import math

from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM

from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error

from time import time

In [182]:
# fix random seed for reproducibility
np.random.seed(7)

In [183]:
# READ FILE
path = '/Users/cathytol/Documents/DMT/DMT/'
inputPath = path + "out_without_nan_mood_target.csv"
df = pd.read_csv(inputPath)
df['date'] = pd.to_datetime(df['date'])


In [184]:
df.head()

,id,date,activity_mean,circumplex.arousal_mean,circumplex.valence_mean,mood_mean,appCat.builtin_sum,appCat.communication_sum,appCat.entertainment_sum,appCat.finance_sum,...,appCat.travel_sum_time_5,appCat.unknown_sum_time_5,appCat.utilities_sum_time_5,appCat.weather_sum_time_5,call_sum_time_5,screen_sum_time_5,sms_sum_time_5,day_time_5,weekDay_time_5,mood_mean_TARGET
0,AS14.01,2014-02-25,0.000000,0.5000,0.384615,NaN,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.0,0.215385,0.000000,0.152174,0.040000,0.625000,6.250000
1,AS14.01,2014-02-26,0.000000,0.4375,0.615385,6.25,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.0,0.123077,0.000000,0.152174,0.052727,0.625000,6.333333
2,AS14.01,2014-03-20,0.144034,0.5000,0.384615,NaN,0.006174,0.104651,0.009763,0.000000,...,0.000000,0.012146,0.004709,0.0,0.046154,0.025445,0.130435,0.263636,0.500000,6.200000
3,AS14.01,2014-03-21,0.236766,0.5500,0.446154,6.20,0.077850,0.303150,0.028033,0.037502,...,0.067479,0.012146,0.138491,0.0,0.138462,0.226449,0.065217,0.272727,0.416667,6.400000
4,AS14.01,2014-03-22,0.418390,0.6500,0.538462,6.40,0.018139,0.239538,0.002597,0.015953,...,0.070228,0.012146,0.164772,0.0,0.184615,0.295118,0.065217,0.281818,0.625000,6.800000


In [185]:
df = df.dropna(subset = ['mood_mean','mood_mean_TARGET'] )

In [186]:
# Create test and train set
length_test = round(len(df)*0.33)
index_test = np.random.choice(np.arange(len(df)), length_test, False)

test = df.iloc[index_test]
train = df.drop(df.index[index_test])

In [187]:
train.head()

,id,date,activity_mean,circumplex.arousal_mean,circumplex.valence_mean,mood_mean,appCat.builtin_sum,appCat.communication_sum,appCat.entertainment_sum,appCat.finance_sum,...,appCat.travel_sum_time_5,appCat.unknown_sum_time_5,appCat.utilities_sum_time_5,appCat.weather_sum_time_5,call_sum_time_5,screen_sum_time_5,sms_sum_time_5,day_time_5,weekDay_time_5,mood_mean_TARGET
1,AS14.01,2014-02-26,0.000000,0.4375,0.615385,6.25,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.00000,0.123077,0.000000,0.152174,0.052727,0.625000,6.333333
3,AS14.01,2014-03-21,0.236766,0.5500,0.446154,6.20,0.077850,0.303150,0.028033,0.037502,...,0.067479,0.012146,0.138491,0.00000,0.138462,0.226449,0.065217,0.272727,0.416667,6.400000
4,AS14.01,2014-03-22,0.418390,0.6500,0.538462,6.40,0.018139,0.239538,0.002597,0.015953,...,0.070228,0.012146,0.164772,0.00000,0.184615,0.295118,0.065217,0.281818,0.625000,6.800000
7,AS14.01,2014-03-25,0.173753,0.6250,0.538462,6.75,0.025612,0.496012,0.001898,0.032589,...,0.101173,0.063247,0.256523,0.05825,0.292308,0.753727,0.043478,0.309091,0.666667,6.600000
9,AS14.01,2014-03-27,0.281736,0.5500,0.630769,7.00,0.030486,0.334794,0.017790,0.031957,...,0.034432,0.063247,0.143916,0.05825,0.184615,0.862674,0.043478,0.327273,0.500000,6.400000


In [51]:
#list(train.columns)

In [188]:
#test[['id','date','mood_mean', 'mood_mean_TARGET']].head(40)

In [255]:
def create_dataset(dataset, look_back=1):
    dataX, dataY = [], []
    for name, group in dataset.groupby('id'):
        x = list(group['mood_mean'])
        #dataX, dataY = [], []
        for i in range(len(group)-look_back):
            #print(i)
            dataX.append(x[i:(i+look_back)])
            dataY.append(x[i + look_back])
    return np.array(dataX), np.array(dataY)


look_back = 3
trainX, trainY = create_dataset(train[['id', 'mood_mean']], look_back)
testX, testY = create_dataset(test[['id', 'mood_mean']], look_back)


In [53]:
""" 
trainX = np.array([train['mood_mean']])
trainY = np.array([train['mood_mean_TARGET']])

testX = np.array([test['mood_mean']])
testY = np.array([test['mood_mean_TARGET']])
"""

In [256]:
# reshape input to be [samples, time steps, features]
trainX = np.reshape(trainX, (trainX.shape[0], 1, trainX.shape[1]))
testX = np.reshape(testX, (testX.shape[0], 1, testX.shape[1]))

In [257]:
trainX.shape

(734, 1, 3)

In [ ]:
# define model
model = Sequential()
model.add(LSTM(50, activation='relu', input_shape=(n_steps, n_features)))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mse')

In [196]:
t = time()

# create and fit the LSTM network
model = Sequential()
model.add(LSTM(4, input_shape=(1, look_back)))
model.add(Dense(1))
model.compile(loss='mean_squared_error', optimizer='adam')
model.fit(trainX, trainY, epochs=10, batch_size=1, verbose=2)

print('Time to train the model: {} mins'.format(round((time() - t) / 60, 2)))

Epoch 1/10
 - 2s - loss: 35.1151
Epoch 2/10
 - 1s - loss: 9.0718
Epoch 3/10
 - 1s - loss: 2.6489
Epoch 4/10
 - 1s - loss: 0.8697
Epoch 5/10
 - 1s - loss: 0.5642
Epoch 6/10
 - 1s - loss: 0.5337
Epoch 7/10
 - 1s - loss: 0.5101
Epoch 8/10
 - 1s - loss: 0.4854
Epoch 9/10
 - 1s - loss: 0.4790
Epoch 10/10
 - 1s - loss: 0.4746
Time to train the model: 0.24 mins


In [258]:
t = time()

# make predictions
trainPredict = model.predict(trainX)
testPredict = model.predict(testX)

# calculate root mean squared error
trainScore = math.sqrt(mean_squared_error(trainY, trainPredict[:,0]))
print('Train Score: %.3f RMSE' % (trainScore))
testScore = math.sqrt(mean_squared_error(testY, testPredict[:,0]))
print('Test Score: %.3f RMSE' % (testScore))

print('Time to predict: {} mins'.format(round((time() - t) / 60, 2)))

Train Score: 0.685 RMSE
Test Score: 0.723 RMSE
Time to predict: 0.0 mins


In [265]:
# Create empty df to save solutions
solutions = pd.DataFrame(data={'id': [], 'date': [], 'mood_mean_target': [], 'predicted_mood_mean_target':[]})

In [266]:
# Save solutions in df 
j = 0
for name, group in train.groupby('id'):
    for i in range(look_back, len(group)):
        #print(j, i)
        solutions = solutions.append({'id': group.iloc[i]['id'], 'date': group.iloc[i]['date'], 'mood_mean_target': group.iloc[i]['mood_mean_TARGET'],
                                      'predicted_mood_mean_target': trainPredict[j]}, ignore_index=True)
        j += 1    

In [267]:
def calculateAccMSQ(solutions, name_target,name_predicted):
    correct = 0
    squared_error = 0

    for index, row in solutions.iterrows():
        squared_error += math.pow(row[name_target]-row[name_predicted],2)
        if (row[name_target] == row[name_predicted]):
            correct = correct + 1
        
    accuracy = correct/ len(solutions.index)
    msq = squared_error / len(solutions.index)
    return accuracy, msq

In [271]:
accuracy, msq = calculateAccMSQ(solutions, "mood_mean_target", "predicted_mood_mean_target")
print(accuracy, msq)

0.0 0.49413052027054255


In [272]:
math.sqrt(msq)

0.7029441800531124

In [273]:
solutions2 = solutions.round({'mood_mean_target': 0, 'predicted_mood_mean_target': 0})
accuracy, msq = calculateAccMSQ(solutions2, "mood_mean_target", "predicted_mood_mean_target")
print(accuracy, msq)

0.0 0.6076599588963617


In [274]:
math.sqrt(msq)

0.779525470332023